<h1><b>SCRAPING-DATA-SEKOLAH</b></h1>

---

📝 **Notebook: 2_scraping_profil.ipynb**

Notebook ini digunakan untuk **scraping data profil sekolah secara lebih detail** berdasarkan hasil dari scraping sebelumnya (`semua_<jenjang>.csv`).  

---

📂 **Sumber Data**

Data input yang digunakan berasal dari hasil scraping sebelumnya:  
- **`../hasil/semua_<jenjang>.csv`** → berisi metadata dasar sekolah (nama, alamat, provinsi, kabupaten/kota, koordinat, dsb.)  
> ⚠️ CATATAN: Data ini **harus diupload terlebih dahulu ke Google Spreadsheet** (Worksheet: `scraping-profil`) sehingga progres scraping dapat dipantau secara **real-time**.  
> Spreadsheet dapat diakses di [data_semua_sma](https://docs.google.com/spreadsheets/d/1eJ-uylcTjo_OGhcih7pgacc28KCz9Y8bx6tNw1157GM).

Dataset ini akan diperluas dengan informasi tambahan dari halaman profil masing-masing sekolah, seperti akreditasi, profil sekolah, fasilitas, dan informasi terkait lainnya.

---

⚙️ **Tahapan Scraping Profil**

Beberapa langkah yang dilakukan di notebook ini:  

1. **LIBRARY** 
- Import library yang dibutuhkan untuk scraping dan manipulasi data.

2. **CONFIGURATION**
- Menyiapkan Google Spreadsheet API untuk sinkronasi data input dan output (hasil scraping profil)

3. **SCRAPING**  
- Mengambil data awal (**scraping halaman**) dari Google Spreadsheet untuk kemudian diambil data detailnya (**scraping profil**).  
- Terdapat banyak fungsi ekstraksi karena banyak atribut yang dikumpulkan.  
- Scraping dilakukan secara paralel untuk semua sekolah, dan hasilnya langsung terupdate pada Google Spreadsheet (Worksheet: `scraping-maps`)  
  > - File: [data_semua_sma](https://docs.google.com/spreadsheets/d/1eJ-uylcTjo_OGhcih7pgacc28KCz9Y8bx6tNw1157GM)

---

📊 **Hasil yang Diperoleh**

Proses scraping ini akan menghasilkan:  

- Dataset sekolah yang lebih lengkap dengan profil detail  
- Informasi siap digunakan untuk **analisis lebih lanjut dan scraping citra satelit**  
- File CSV siap digunakan untuk tahap berikutnya **Scraping Citra Satelit** 

Output utama berupa dataset akan disimpan di Google Spreadsheet:  
📂 **[Data Semua SMA – Worksheet: scraping-profil](https://docs.google.com/spreadsheets/d/1eJ-uylcTjo_OGhcih7pgacc28KCz9Y8bx6tNw1157GM)**  

---


---
# **LIBRARY**
---

Di bagian ini, kita mengimpor library yang diperlukan untuk:  
- Manipulasi file dan data (`os`, `glob`, `pandas`)  
- Operasi matematika (`math`)  
- Scraping web (`requests`, `BeautifulSoup`)  
- Progress monitoring (`tqdm`)  
- Google Sheets / OAuth (`gspread`, `oauth2client`)  
- Concurrency / multi-threading (`ThreadPoolExecutor`, `as_completed`)


In [ ]:

# ============================
# Standard Library
# ============================
import os               # Operasi sistem (buat folder, path, dll)
import math             # Fungsi matematika (ceil, floor, dll)
import glob             # Operasi file dengan pola tertentu
import pandas as pd     # Manipulasi data tabel (DataFrame)
import json

# ============================
# Google Sheets / OAuth
# ============================
import gspread                                                       # Mengakses google spreadsheets
from oauth2client.service_account import ServiceAccountCredentials 

# ============================
# Third-party Libraries
# ============================
import requests                # HTTP request (scraping data dari web)
from bs4 import BeautifulSoup  # Parsing HTML
from tqdm import tqdm          # Progress bar interaktif

# ============================
# Concurrency Utilities
# ============================
from concurrent.futures import ThreadPoolExecutor, as_completed  # Multi-worker/concurrent

---
# **CONFIGURATION**
---

Berbeda dengan proses scraping sebelumnya, pada proses ini **file akan disimpan dan disinkronkan menggunakan Google Sheets**, sehingga progres scraping dapat dipantau secara **real-time**.  

Sebelum menjalankan notebook ini, ada beberapa hal yang perlu disiapkan:

1. **File Google Spreadsheet** yang dapat diakses di [data_semua_sma](https://docs.google.com/spreadsheets/d/1eJ-uylcTjo_OGhcih7pgacc28KCz9Y8bx6tNw1157GM).  
2. **Service Account** dengan akses API Google Spreadsheet, diperoleh melalui **Google Cloud Console**.  
   > ⚠️ Untuk keamanan, credentials service account **tidak dicantumkan** di notebook.

Bagian ini berisi fungsi-fungsi untuk:  
1. `auth(data)` → Mengautentikasi dan mengembalikan client gspread.  
2. `start(data, id_sheet)` → Membuka worksheet 'scrapping-profil' dari spreadsheet.  
3. `ambil_baris(sheet, i)` → Mengambil nilai dari baris ke-i pada worksheet.  
4. `simpan(sheet, data, i)` → Menyimpan/update data ke baris ke-i pada worksheet. 
> ⚠️ Kolom yang digunakan dalam spreadsheet harus sesuai format yang ditentukan (misal 233 kolom: I–IG).

In [ ]:
# ============================
# Fungsi autentikasi
# ============================
def auth(data):
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive"
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_dict(data, scope)
    client = gspread.authorize(creds)
    return client

# ============================
# Fungsi mulai akses spreadsheet
# ============================
def start(data, id_sheet):
    client = auth(data)
    sheet = client.open_by_key(id_sheet).worksheet('scraping-profil') # Nama Worksheet
    return client, sheet

# ============================
# Fungsi mengambil data baris
# ============================
def ambil_baris(sheet, i):
    try:
        return sheet.row_values(i+4) # + 4 karena data dimulai pada baris ke 4 (3 baris pertama merupakan header)
    except Exception as e:
        print(f"Error ketika mengambil data baris ke-{i}: {e}")
        return 0

# ============================
# Fungsi menyimpan data
# ============================
def simpan(sheet, data, i):
    if len(data) != 233:
        raise ValueError("Data harus berisi tepat 233 elemen (I–IG)")
    
    cell_range = f"I{i+4}:IG{i+4}"
    
    try:
        sheet.update(range_name=cell_range, values=[data])
    except Exception as e:
        print(f"Error ketika mengupdate data baris ke-{i}: {e}")

In [ ]:
# ID dari Google spreadsheets yang digunakan
id_sma = '1eJ-uylcTjo_OGhcih7pgacc28KCz9Y8bx6tNw1157GM' # Berisi file SMA

# Berikut ini adalah data Service Account 
laptop_aziz = {
    # Rahasia, ganti dengan service account saudara sendiri.
}

# Mengoneksikan dengan Google Spreadsheets
client, sheet = start(data=laptop_aziz, id_sheet=id_sma)

---
# **SCRAPING**
---

## Fungsi

Fungsi Scraping Profil Sekolah

Berisi fungsi-fungsi untuk mengambil **data profil sekolah secara detail** dari halaman profil sekolah. Data yang diambil mencakup:  

- Informasi umum sekolah: akreditasi, kepala sekolah, operator  
- Profil statistik: guru, siswa, rombongan belajar, sarana-prasarana, kurikulum, manajemen, semester data  
- Ekstrak tambahan: akreditasi, ekstrakurikuler, bantuan, sanitasi, instansi dan sekolah terdekat  
- Memproses data menjadi **format flatten** agar mudah disimpan ke file CSV atau spreadsheet  
- Menyimpan hasil scraping ke **JSON** dengan struktur per sekolah  

| Fungsi | Deskripsi |
|--------|-----------|
| `get_detail_sekolah(html_soup)` | Mengambil informasi **akreditasi, kepala sekolah, operator** |
| `get_detail_profil(html_soup)` | Mengambil data **guru, siswa, rombongan belajar, sarana-prasarana, akses internet, daya listrik, luas tanah, ruang kelas, laboratorium, perpustakaan, sanitasi** |
| `get_detail_ekskul(html_soup)` | Mengambil data **rasio siswa, kualifikasi guru, kondisi ruang kelas** terkait ekstrakurikuler |
| `get_instansi_dan_sekolah(html_soup)` | Mengambil **instansi terdekat dan sekolah terdekat** beserta jaraknya |
| `get_detail_akreditasi(html_soup)` | Mengambil informasi detail akreditasi seperti **standar, tahun, nilai akhir** |
| `get_detail_bantuan(html_soup)` | Mengambil data **bantuan, rehab, BOS, PIP, TIK, sarana kesenian** |
| `get_detail_sanitasi(html_soup)` | Mengambil data **sanitasi sekolah** seperti sumber air, toilet, tempat cuci tangan |
| `get_detail(html_soup)` | Mengambil tabel-tabel terstruktur untuk **siswa, guru, tenaga kependidikan, sarana-prasarana** dalam format yang siap diolah |
| `flatten_values(d)` | Mengubah **nested dict/list** menjadi list flat agar mudah disimpan ke CSV atau spreadsheet |
| `scrapper(i, nama, url, tipe_sekolah)` | Fungsi utama untuk **scraping profil sekolah**, menggabungkan semua fungsi di atas, memanggil halaman web, mengekstrak data, menyimpan JSON, dan mengembalikan data **flatten** |


In [ ]:
# ==========================
# File HTML dummy untuk pengecekan keluaran fungsi
# ==========================
file_sekolah = "dummy/nama_sekolah_sekolah.html"
file_sanitasi = "dummy/nama_sekolah_sanitasi.html"

def load_html_as_soup(file_path):
    """Membaca file HTML dan mengembalikan BeautifulSoup object"""
    with open(file_path, "r", encoding="utf-8") as f:
        return BeautifulSoup(f.read(), "html.parser")

# Load HTML
soup_sekolah = load_html_as_soup(file_sekolah)
soup_sanitasi = load_html_as_soup(file_sanitasi)


`get_detail_sekolah()` 
<br><br>
<img src="gambar/get_detail_sekolah.png" width="800"/>


In [ ]:
def get_detail_sekolah(html_soup, debug=False):
    # Mengambil semua "baris" (<li>) dari "tabel" (<ul>) dengan class 'list-group'
    items = html_soup.select("ul.list-group li.list-group-item")
    
    # print(items) # Un-comment jika ingin tau isi dari items

    data = {
        "akreditasi": None,
        "kepala_sekolah": None,
        "operator": None
    }

    for li in items:
        text = li.get_text(" ", strip=True)
        
        # Jika barisnya merupakan "Akreditasi"
        if "Akreditasi" in text:        
            data["akreditasi"] = text.split(":", 1)[-1].strip() if ":" in text else None
        
        # Jika barisnya merupakan "Kepala Sekolah"
        elif "Kepala Sekolah" in text:  
            data["kepala_sekolah"] = text.split(":", 1)[-1].strip() if ":" in text else None
            
        # Jika barisnya merupakan "Operator"
        elif "Operator" in text:
            link = li.find("a")
            data["operator"] = link.get_text(strip=True) if link else text.split(":", 1)[-1].strip()
    
    if debug:
        print(f"items: {items}")

    return data

In [ ]:
get_detail_sekolah(soup_sekolah, debug=True)

items: [<li class="list-group-item bg-blue"><a href="http://data.dikdasmen.kemdikbud.go.id/sp/profile/00BEECB3-074E-4C4A-A882-A60ADD900D8A" style="color:white" target="_blank"><i class="glyphicon glyphicon-info-sign"></i> Detail Sekolah (DAPODIK) <small>Klik Disini</small></a></li>, <li class="list-group-item list-group-item-info"><i class="glyphicon glyphicon-font"></i> <b>Akreditasi</b> : A</li>, <li class="list-group-item"><i class="glyphicon glyphicon-user"></i>  Kepala Sekolah : Mustajab</li>, <li class="list-group-item"><i class="fa fa-user"></i>  Operator : <a href="https://sdm.data.kemdikbud.go.id/instansi/view?id=00BEECB3-074E-4C4A-A882-A60ADD900D8A" style="color:#67B1D6" target="_blank">Seri Ayuna</a></li>, <li class="list-group-item bg-gray"><a href="http://ekskul.data.kemdikbud.go.id/index.php/cpublic/profil?npsn=10100139" target="_blank"><b>Ekstrakurikuler dan Muatan Lokal</b></a></li>, <li class="list-group-item bg-blue"><b>Proses Pembelajaran</b></li>, <li class="list-gr

{'akreditasi': 'A', 'kepala_sekolah': 'Mustajab', 'operator': 'Seri Ayuna'}

`get_detail_profil()` 
<br><br>
<img src="gambar/get_detail_profil.png" width="800"/>

In [ ]:
def get_detail_profil(html_soup, debug=False):
    data = {
        "guru": None,
        "siswa_l": None,
        "siswa_p": None,
        "rombongan_belajar": None,
        "kurikulum": None,
        "penyelenggaraan": None,
        "manajemen_berbasis_sekolah": None,
        "semester_data": None,
        "akses_internet": None,
        "sumber_listrik": None,
        "daya_listrik": None,
        "luas_tanah": None,
        "ruang_kelas": None,
        "laboratorium": None,
        "perpustakaan": None,
        "sanitasi_siswa": None
    }

    def safe_text(el):
        return el.get_text(strip=True) if el else None

    blocks = html_soup.select("div.col-md-3")

    # Guru, Siswa, Rombel
    fonts = blocks[0].select("font.text-info")
    data["guru"], data["siswa_l"], data["siswa_p"], data["rombongan_belajar"] = [safe_text(f) for f in fonts]

    # Penyelenggaraan, MBS, Semester
    block2 = blocks[1]
    penyelenggaraan_font = block2.select_one("i.glyphicon-globe + font.text-info")
    data["penyelenggaraan"] = safe_text(penyelenggaraan_font)

    data["manajemen_berbasis_sekolah"] = "Ya" if block2.find("i", class_="glyphicon glyphicon-check") else "Tidak"
    data["semester_data"] = safe_text(block2.select_one("font.text-info"))

    # Akses Internet, Sumber Listrik, Daya Listrik, Luas Tanah
    block3 = blocks[2]
    data["akses_internet"] = safe_text(block3.select_one("i.glyphicon-signal + font.text-info"))
    data["sumber_listrik"] = safe_text(block3.select_one("i.glyphicon-flash + font.text-info"))
    data["daya_listrik"] = "Ya" if block3.find("i", class_="glyphicon glyphicon-check") else (
                           "Tidak" if block3.find("i", class_="glyphicon glyphicon-unchecked") else None)
    luas_tanah_font = block3.select_one("i.glyphicon-globe + font.text-info")
    data["luas_tanah"] = safe_text(luas_tanah_font)
    data["luas_tanah"] = str(data["luas_tanah"]).replace(",", "")

    # Ruang Kelas, Lab, Perpus, Sanitasi
    fonts4 = blocks[3].select("font.text-info")
    data["ruang_kelas"], data["laboratorium"], data["perpustakaan"], data["sanitasi_siswa"] = [safe_text(f) for f in fonts4]

    if debug:
        print(f"blocks: {blocks}")
    return data

In [ ]:
get_detail_profil(soup_sekolah, debug=True)

blocks: [<div class="col-xs-12 col-md-3 text-left" style="border-right: 2px solid #31708f">
<i class="glyphicon glyphicon-user"></i> Guru : <font class="text-info">0</font><br/>
<i class="fa fa-user"></i>  Siswa Laki-laki : <font class="text-info">89</font><br/>
<i class="fa fa-user"></i>  Siswa Perempuan : <font class="text-info">82</font><br/>
<i class="glyphicon glyphicon-list-alt"></i> Rombongan Belajar : <font class="text-info">6</font><br/>

						 

					</div>, <div class="col-xs-12 col-md-3 text-left" style="border-right: 2px solid #31708f">
<i class="glyphicon glyphicon-education"></i> Kurikulum : <font class="text-info"> SMA Merdeka</font><br/>
<i class="glyphicon glyphicon-globe"></i> Penyelenggaraan : Pagi/6 hari<br/>
<i class="glyphicon glyphicon-cog"></i> Manajemen Berbasis Sekolah : <i class="glyphicon glyphicon-check text-info"></i><br/>
<i class="glyphicon glyphicon-cog"></i> Semester Data : <font class="text-info">2025/2026-1</font><br/>

						 

					</div>, <div c

{'guru': '0',
 'siswa_l': '89',
 'siswa_p': '82',
 'rombongan_belajar': '6',
 'kurikulum': None,
 'penyelenggaraan': None,
 'manajemen_berbasis_sekolah': 'Tidak',
 'semester_data': 'SMA Merdeka',
 'akses_internet': None,
 'sumber_listrik': '0',
 'daya_listrik': None,
 'luas_tanah': '15030',
 'ruang_kelas': '0',
 'laboratorium': '0',
 'perpustakaan': '0',
 'sanitasi_siswa': '2'}

`get_detail_ekskul()` 
<br><br>
<img src="gambar/get_detail_ekskul.png" width="800"/>

In [ ]:
def get_detail_ekskul(html_soup, debug=False):
    data = {
        "rasio_siswa_rombel": None,
        "rasio_siswa_ruang_kelas": None,
        "rasio_siswa_guru": None,
        "persentase_guru_kualifikasi": None,
        "persentase_guru_sertifikasi": None,
        "persentase_guru_pns": None,
        "persentase_ruang_kelas_layak": None
    }

    mapping = {
        "Rasio Siswa Rombel": "rasio_siswa_rombel",
        "Rasio Siswa Ruang Kelas": "rasio_siswa_ruang_kelas",
        "Rasio Siswa Guru": "rasio_siswa_guru",
        "Persentase Guru Kualifikasi": "persentase_guru_kualifikasi",
        "Persentase Guru Sertifikasi": "persentase_guru_sertifikasi",
        "Persentase Guru PNS": "persentase_guru_pns",
        "Persentase Ruang Kelas Layak": "persentase_ruang_kelas_layak"
    }
    
    items = html_soup.select("div.col-md-5 li.list-group-item")
    for li in items:
        label_tag = li.select_one("a.text-warning")
        value_tag = li.select_one("span.badge.pull-right")
        if label_tag and value_tag:
            label = label_tag.get_text(strip=True).replace("*", "").strip()
            if label in mapping:
                data[mapping[label]] = value_tag.get_text(strip=True)

    if debug:
        print(f"items: {items}")
    return data

In [ ]:
get_detail_ekskul(soup_sekolah, debug=True)

items: [<li class="list-group-item bg-gray"><a href="http://ekskul.data.kemdikbud.go.id/index.php/cpublic/profil?npsn=10100139" target="_blank"><b>Ekstrakurikuler dan Muatan Lokal</b></a></li>, <li class="list-group-item bg-blue"><b>Proses Pembelajaran</b></li>, <li class="list-group-item"><a class="text-warning">Rasio Siswa Rombel</a><span class="badge pull-right">28.5</span></li>, <li class="list-group-item"><a class="text-warning">Rasio Siswa Ruang Kelas <font color="red">*</font></a><span class="badge pull-right">0</span></li>, <li class="list-group-item"><a class="text-warning">Rasio Siswa Guru</a><span class="badge pull-right">0</span></li>, <li class="list-group-item"><a class="text-warning">Persentase Guru Kualifikasi</a><span class="badge pull-right">0</span></li>, <li class="list-group-item"><a class="text-warning">Persentase Guru Sertifikasi</a><span class="badge pull-right">0</span></li>, <li class="list-group-item"><a class="text-warning">Persentase Guru PNS</a><span class

{'rasio_siswa_rombel': '28.5',
 'rasio_siswa_ruang_kelas': '0',
 'rasio_siswa_guru': '0',
 'persentase_guru_kualifikasi': '0',
 'persentase_guru_sertifikasi': '0',
 'persentase_guru_pns': '0',
 'persentase_ruang_kelas_layak': '0'}

`get_instansi_dan_sekolah()` 
<br><br>
<img src="gambar/get_instansi_dan_sekolah.png" width="800"/>

In [ ]:
def get_instansi_dan_sekolah(html_soup, debug=False):
    data = {
        "instansi_terdekat": None,
        "sekolah_terdekat": None
    }

    # Ambil instansi
    instansi_list = []
    items_instanasi = html_soup.select("li.list-group-item.list-group-item-info.small")
    for li in items_instanasi:
        link = li.find("a")
        jarak_tag = li.find("em")
        if link and jarak_tag:
            nama = link.get_text(strip=True)
            jarak = jarak_tag.get_text(strip=True)
            instansi_list.append(f"{nama} {jarak}")

    # Ambil sekolah
    sekolah_list = []
    items_sekolah = html_soup.select("li.list-group-item.small:not(.list-group-item-info)")
    for li in items_sekolah:
        link = li.find("a")
        jarak_tag = li.find("em")
        if link and jarak_tag:
            nama = link.get_text(strip=True)
            jarak = jarak_tag.get_text(strip=True)
            sekolah_list.append(f"{nama} {jarak}")

    data['instansi_terdekat'] = ", ".join(instansi_list)
    data['sekolah_terdekat'] = ", ".join(sekolah_list)

    if debug:
        print(f"items instansi: {items_instanasi}")
        print(f"items sekolah: {items_sekolah}")
    return data

In [ ]:
get_instansi_dan_sekolah(soup_sekolah, debug=True)

items instansi: [<li class="list-group-item list-group-item-info small"><i class="glyphicon glyphicon-cog"></i>  <a href="http://sdm.data.kemdikbud.go.id/index.php?r=instansi/view&amp;id=F9A9518B-E25E-4C36-AD81-764D2CB78312" target="_blank">Dinas Pendidikan Provinsi Aceh</a>  <em>(44.76 Km)</em><br/>Jl. Tgk. H. Mohd. Daud Beureueh No.22 Kuta Alam, B ...</li>, <li class="list-group-item list-group-item-info small"><i class="glyphicon glyphicon-cog"></i>  <a href="http://sdm.data.kemdikbud.go.id/index.php?r=instansi/view&amp;id=763C1D95-DCB7-47C3-B70C-2208205A4625" target="_blank">Dinas Pendidikan dan Kebudayaan Kab. Aceh Besar</a>  <em>(11.13 Km)</em><br/>Jl. T. Bakhtiar No.10 Panglima Polim SH-Kota Janth ...</li>, <li class="list-group-item list-group-item-info small"><i class="glyphicon glyphicon-cog"></i>  <a href="http://sdm.data.kemdikbud.go.id/index.php?r=instansi/view&amp;id=5B9824EB-2524-4877-A57E-6F027B3F19A4" target="_blank">Balai Bahasa Aceh</a>  <em>(43.4 Km)</em><br/>Jalan 

{'instansi_terdekat': 'Dinas Pendidikan Provinsi Aceh (44.76 Km), Dinas Pendidikan dan Kebudayaan Kab. Aceh Besar (11.13 Km), Balai Bahasa Aceh (43.4 Km)',
 'sekolah_terdekat': 'SMAIT Nurul Fikri Boarding School Aceh (8.39), SMA NEGERI 1 KOTA JANTHO (12.48), SMA NEGERI 1 SEULIMEUM (12.83), PKBM Pante Leungah (10.62 Km), PKBM AL-ITTIHADIYAH (10.62 Km), SKB ACEH BESAR (12.3 Km)'}

`get_detail_akreditasi()` 
<br><br>
<img src="gambar/get_detail_akreditasi.png" width="800"/>

In [ ]:
def get_detail_akreditasi(html_soup, debug=False):
    data = {
        "standar_isi": None,
        "standar_proses": None,
        "standar_kelulusan": None,
        "standar_tenaga_pendidik": None,
        "standar_sarana_prasarana": None,
        "standar_pengelolaan": None,
        "standar_pembiayaan": None,
        "standar_penilaian": None,
        "tahun": None,
        "nilai_akhir": None,
        "akreditasi": None
    }

    mapping = {
        "Standar Isi": "standar_isi",
        "Standar Proses": "standar_proses",
        "Standar Kelulusan": "standar_kelulusan",
        "Standar Tenaga Pendidik": "standar_tenaga_pendidik",
        "Standar Sarana Prasarana": "standar_sarana_prasarana",
        "Standar Pengelolaan": "standar_pengelolaan",
        "Standar Pembiayaan": "standar_pembiayaan",
        "Standar Penilaian": "standar_penilaian",
        "Tahun": "tahun",
        "Nilai Akhir": "nilai_akhir",
        "Akreditasi": "akreditasi"
    }

    items = html_soup.select("#dataakreditasi li.list-group-item")
    for li in items:
        text = li.get_text(strip=True)
        if ":" in text:
            label, value = text.split(":", 1)
            label = label.strip()
            value = value.strip()
            if label in mapping:
                data[mapping[label]] = value

    if debug:
        print(f"items: {items}")
    return data

In [ ]:
get_detail_akreditasi(soup_sekolah, debug=True)

items: [<li class="list-group-item text-muted"><i class="glyphicon glyphicon-ok-sign text-info"></i>  Standar Isi : </li>, <li class="list-group-item text-muted"><i class="glyphicon glyphicon-ok-sign text-info"></i>  Standar Proses : </li>, <li class="list-group-item text-muted"><i class="glyphicon glyphicon-ok-sign text-info"></i>  Standar Kelulusan : </li>, <li class="list-group-item text-muted"><i class="glyphicon glyphicon-ok-sign text-info"></i>  Standar Tenaga Pendidik : </li>, <li class="list-group-item text-muted"><i class="glyphicon glyphicon-ok-sign text-info"></i>  Standar Sarana Prasarana : </li>, <li class="list-group-item text-muted"><i class="glyphicon glyphicon-ok-sign text-info"></i>  Standar Pengelolaan : </li>, <li class="list-group-item text-muted"><i class="glyphicon glyphicon-ok-sign text-info"></i>  Standar Pembiayaan : </li>, <li class="list-group-item text-muted"><i class="glyphicon glyphicon-ok-sign text-info"></i>  Standar Penilaian : </li>, <li class="list-g

{'standar_isi': '',
 'standar_proses': '',
 'standar_kelulusan': '',
 'standar_tenaga_pendidik': '',
 'standar_sarana_prasarana': '',
 'standar_pengelolaan': '',
 'standar_pembiayaan': '',
 'standar_penilaian': '',
 'tahun': '2024',
 'nilai_akhir': '0',
 'akreditasi': 'A'}

`get_detail_bantuan()` 
<br><br>
<img src="gambar/get_detail_bantuan.png" width="800"/>

In [ ]:
def get_detail_bantuan(html_soup, debug=False):
    data = {
        "rehab": {
            "Tahun": "",
            "Jumlah Ruang": ""
        },
        "revitalisasi": {
            "Tahun": "",
            "Jenis": "",
            "Pembina": ""
        },
        "rkb": {
            "Tahun": "",
            "Jenis": "",
            "Pembina": ""
        },
        "sanitasi": {
            "Tahun": "",
            "Jenis": "",
            "Pembina": ""
        },
        "usb": {
            "Tahun": "",
            "Jenis": "",
            "Pembina": ""
        },
        "bos": {
            "Tahun": "",
            "Jumlah Siswa": ""
        },
        "pip": {
            "Tahun": "",
            "Jumlah Penerima": ""
        },
        "tik": {
            "Tahun": "",
            "Jenis": "",
            "Pembina": ""
        },
        "sarana_kesenian": {
            "Tahun": "",
            "Pembina": ""
        },
    }

    mapping = {
        "rehab": "rehab",
        "revitalisasi": "revitalisasi",
        "rkb": "rkb",
        "sanit": "sanitasi",
        "usb": "usb",
        "bos": "bos",
        "pip": "pip",
        "tik": "tik",
        "saranakesenian": "sarana_kesenian",
        "vokasi": "vokasi"
    }

    for tab_id, key in mapping.items():
        table = html_soup.select_one(f"#{tab_id} table")
        if not table:
            continue

        # Ambil header kolom
        headers = [th.get_text(strip=True) for th in table.select("tr th")]
        # Ambil baris pertama data (kalau ada)
        rows = table.select("tr")[1:]
        if rows:
            first_row = [td.get_text(strip=True) for td in rows[0].select("td")]
            for h, v in zip(headers, first_row):
                data[key][h] = v
        
        if debug:
            print(f"Tabel {tab_id}: {table}")

    return data


In [ ]:
get_detail_bantuan(soup_sekolah, debug=True)

Tabel rehab: <table border="1" bordercolor="#CCCCCC" class="table table-hover">
<tr>
<th class="text-center">Tahun</th>
<th class="text-center">Jumlah Ruang</th>
</tr>
</table>
Tabel revitalisasi: <table border="1" bordercolor="#CCCCCC" class="table table-hover">
<tr>
<th class="text-center">Tahun</th>
<th class="text-center">Jenis</th>
<th class="text-center">Pembina</th>
</tr>
</table>
Tabel rkb: <table border="1" bordercolor="#CCCCCC" class="table table-hover">
<tr>
<th class="text-center">Tahun</th>
<th class="text-center">Jenis</th>
<th class="text-center">Pembina</th>
</tr>
</table>
Tabel sanit: <table border="1" bordercolor="#CCCCCC" class="table table-hover">
<tr>
<th class="text-center">Tahun</th>
<th class="text-center">Jenis</th>
<th class="text-center">Pembina</th>
</tr>
</table>
Tabel usb: <table border="1" bordercolor="#CCCCCC" class="table table-hover">
<tr>
<th class="text-center">Tahun</th>
<th class="text-center">Jenis</th>
<th class="text-center">Pembina</th>
</tr>
<

{'rehab': {'Tahun': '', 'Jumlah Ruang': ''},
 'revitalisasi': {'Tahun': '', 'Jenis': '', 'Pembina': ''},
 'rkb': {'Tahun': '', 'Jenis': '', 'Pembina': ''},
 'sanitasi': {'Tahun': '', 'Jenis': '', 'Pembina': ''},
 'usb': {'Tahun': '', 'Jenis': '', 'Pembina': ''},
 'bos': {'Tahun': '2013', 'Jumlah Siswa': '197'},
 'pip': {'Tahun': '2015', 'Jumlah Penerima': '88'},
 'tik': {'Tahun': '', 'Jenis': '', 'Pembina': ''},
 'sarana_kesenian': {'Tahun': '2015', 'Pembina': 'Direktorat Kesenian'}}

`get_detail_sanitasi()` 
<br><br>
<div style="display: flex; gap: 10px;">
  <img src="gambar/get_detail_sanitasi_1.png" width="600">
  <img src="gambar/get_detail_sanitasi_2.png" width="600">
</div>


In [ ]:

def get_detail_sanitasi(html_soup, debug=False):
    data = {}

    # Ambil semua <td> dalam tabel
    items = html_soup.select("table.table-bordered td")
    for td in items:
        text = td.get_text(strip=True)
        if ":" in text:
            key, value = text.split(":", 1)
            data[key.strip()] = value.strip()

    if data == {}:
        data = {
            "Sumber Air": "",
            "Kecukupan Air": "",
            "Ketersediaan Air Minum Siswa": "",
            "Memproses Air": "",
            "Siswa Membawa Air": "",
            "Toilet Siswa Laki-laki": "",
            "Toilet Siswa Perempuan": "",
            "Toilet Siswa Umum": "",
            "Toilet Siswa Kecil": "",
            "Tipe Jamban": "",
            "Tempat Cuci Tangan": "",
            "Air Mengalir": ""
        }
    
    if debug:
        print(f"items: {items}")

    return data


In [ ]:
get_detail_sanitasi(soup_sanitasi, debug=True)

items: [<td><b>Sumber Air</b> : Mata air tdk terlindungi</td>, <td><b>Kecukupan Air</b> : 2</td>, <td><b>Ketersediaan Air Minum Siswa</b> : 0</td>, <td><b>Memproses Air</b> : 0</td>, <td><b>Siswa Membawa Air</b> : 1</td>, <td><b>Toilet Siswa Laki-laki</b> : 0</td>, <td><b>Toilet Siswa Perempuan</b> : 0</td>, <td><b>Toilet Siswa Umum</b> : 0</td>, <td><b>Toilet Siswa Kecil</b> : 0</td>, <td><b>Tipe Jamban</b> : 1</td>, <td><b>Tempat Cuci Tangan</b> : 12</td>, <td><b>Air Mengalir</b> : 1</td>]


{'Sumber Air': 'Mata air tdk terlindungi',
 'Kecukupan Air': '2',
 'Ketersediaan Air Minum Siswa': '0',
 'Memproses Air': '0',
 'Siswa Membawa Air': '1',
 'Toilet Siswa Laki-laki': '0',
 'Toilet Siswa Perempuan': '0',
 'Toilet Siswa Umum': '0',
 'Toilet Siswa Kecil': '0',
 'Tipe Jamban': '1',
 'Tempat Cuci Tangan': '12',
 'Air Mengalir': '1'}

`get_detail()` 
<br><br>
<img src="gambar/get_detail.png" width="800"/>

In [ ]:
def get_detail(html_soup):
    TAB_ID_MAPPING = {
        "siswa": {
            "tingkat": "siswatingkat",
            "agama": "siswaagama",
            "umur": "siswaumur",
            "jenis_kelamin": "siswajeniskelamin"
        },
        "siswa_mengulang": {
            "tingkat": "siswamengulangtingkat",
            "umur": "siswamengulangumur",
            "jenis_kelamin": "siswamengulangjeniskelamin"
        },
        "siswa_baru": {
            "umur": "siswabaruumur",
            "jenis_kelamin": "siswabarujeniskelamin"
        },
        "siswa_lulus": {
            "umur": "lulusanumur",
            "jenis_kelamin": "lulusanjeniskelamin"
        },
        "guru": {
            "status": "gurustatus",
            "golongan": "gurugolongan",
            "sertifikasi": "gurusertifikasi",
            "ijazah": "guruijazah",
            "umur": "guruumur",
            "jenis_kelamin": "gurujeniskelamin"
        },
        "tenaga_kependidikan": {
            "status": "ptkstatus",
            "golongan": "ptkgolongan",
            "ijazah": "ptkijazah",
            "umur": "ptkumur",
            "jenis_kelamin": "ptkjeniskelamin"
        },
        "sarana_prasarana": {
            "ruang_kelas": "ruangkelas",
            "laboratorium": "laboratorium",
            "perpustakaan": "perpustakaan",
            "sanitasi":"sanitasi"
        }
    }

    def ekstrak_tabel(tab_id):
        tabel = html_soup.select_one(f"div#{tab_id} table")
        data = []
        if tabel:
            rows = tabel.select("tr")
            for idx, tr in enumerate(rows):
                if idx == 0:
                    continue
                cols = [td.get_text(" ", strip=True) for td in tr.select("td")]
                if cols:
                    data.append(cols)
        return data

    list_tabel_rapi = []
    for kategori, subtabs in TAB_ID_MAPPING.items():
        for subkategori, tab_id in subtabs.items():
            tabel_mentah = ekstrak_tabel(tab_id)

            # geser baris pertama ke paling akhir
            tabel_mentah.append(tabel_mentah.pop(0))

            # format khusus untuk "agama"
            if tab_id == "siswaagama":
                tabel_rapi = {
                    "laki-laki": {},
                    "perempuan": {}
                    }
                for row in tabel_mentah:
                    if len(row) < 3:
                        continue
                    agama = row[0].lower()
                    laki = int(row[1]) if row[1].isdigit() else None
                    perempuan = int(row[2]) if row[2].isdigit() else None
                    tabel_rapi["laki-laki"][agama] = laki
                    tabel_rapi["perempuan"][agama] = perempuan

            elif tab_id == "gurugolongan" or tab_id == "ptkgolongan":
                tabel_rapi = {
                    "I": int(tabel_mentah[0][0]),
                    "II": int(tabel_mentah[0][1]),
                    "III": int(tabel_mentah[0][2]),
                    "IV": int(tabel_mentah[0][3]),
                    "Total": int(tabel_mentah[0][4]),
                    }

            elif tab_id == "laboratorium" or tab_id == "sanitasi":
                # print(len(tabel_mentah))
                # print(tabel_mentah)
                kolom = tabel_mentah[-1]
                # print(kolom)

                kolom.append("total")
                tabel_mentah.pop(-1)
                tabel_mentah.append(tabel_mentah[0])
                tabel_mentah.pop(0)

                tabel_rapi = {}

                for baris in tabel_mentah:
                    if baris[0] not in tabel_rapi:
                        tabel_rapi[baris[0]] = {}

                    for i, k in enumerate(kolom):
                        tabel_rapi[baris[0]][k] = int(baris[i+1])

                # print(tabel_rapi)

            else:
                tabel_rapi = {k.lower(): int(v) for k, v in tabel_mentah}

            list_tabel_rapi.append(tabel_rapi)

    # mapping langsung jadi TAB_TEMPLATE
    TAB_TEMPLATE = {
        "siswa": {
            "siswa": {
                "tingkat": list_tabel_rapi[0],
                "agama": list_tabel_rapi[1],
                "umur": list_tabel_rapi[2],
                "jenis_kelamin": list_tabel_rapi[3],
            },
            "siswa_mengulang": {
                "tingkat": list_tabel_rapi[4],
                "umur": list_tabel_rapi[5],
                "jenis_kelamin": list_tabel_rapi[6],
            },
            "siswa_baru": {
                "umur": list_tabel_rapi[7],
                "jenis_kelamin": list_tabel_rapi[8],
            },
            "siswa_lulus": {
                "umur": list_tabel_rapi[9],
                "jenis_kelamin": list_tabel_rapi[10],
            },
        },
        "guru": {
            "guru": {
                "status": list_tabel_rapi[11],
                "golongan": list_tabel_rapi[12],
                "sertifikasi": list_tabel_rapi[13],
                "ijazah": list_tabel_rapi[14],
                "umur": list_tabel_rapi[15],
                "jenis_kelamin": list_tabel_rapi[16],
            },
            "tenaga_kependidikan": {
                "status": list_tabel_rapi[17],
                "golongan": list_tabel_rapi[18],
                "ijazah": list_tabel_rapi[19],
                "umur": list_tabel_rapi[20],
                "jenis_kelamin": list_tabel_rapi[21],
            }
        },
        "rombongan_belajar": {
            "tingkat": list_tabel_rapi[0]
        },
        "sarana_prasarana": {
            "ruang_kelas": list_tabel_rapi[22],
            "laboratorium": list_tabel_rapi[23],
            "perpustakaan": list_tabel_rapi[24],
            "sanitasi": list_tabel_rapi[25],
        }

    }

    return TAB_TEMPLATE

`flatten_values()`
<br>
<br>
Fungsi ini digunakan untuk meratakan nilai dari struktur data yang kompleks (dictionary / list / nested) menjadi **list satu dimensi**. Hal ini berguna agar data siap disimpan ke **Google Spreadsheet** menggunakan fungsi `simpan()`.

In [ ]:
def flatten_values(d):
    values = []
    if isinstance(d, dict):
        for v in d.values():
            values.extend(flatten_values(v))
    elif isinstance(d, list):
        for item in d:
            values.extend(flatten_values(item))
    else:
        if isinstance(d, (int, float)):
            values.append(d)
        elif isinstance(d, str):
            d_strip = d.strip()
            if d_strip == "":
                values.append("-")
            else:
                # cek apakah string bisa jadi int
                if d_strip.isdigit():
                    values.append(int(d_strip))
                else:
                    # cek apakah string bisa jadi float
                    try:
                        values.append(float(d_strip))
                    except ValueError:
                        values.append(d_strip)
        else:
            values.append(d)
    return values

Fungsi `scrapper()`

Fungsi ini merupakan **fungsi utama** yang menyatukan semua proses scraping sebelumnya untuk mendapatkan data profil sekolah secara lengkap. Fungsi ini:

1. **Validasi input**  
   - Memastikan parameter `i` berupa integer.
   - Menyiapkan `nama_file` dari nama sekolah agar rapi dan konsisten untuk penyimpanan.

2. **Persiapan URL**  
   - `url_sekolah` → halaman profil sekolah.  
   - `url_sanitasi` → halaman profil sanitasi, diubah dari `url_sekolah` secara otomatis.

3. **Ambil halaman HTML**  
   - Menggunakan `requests.get()` dengan timeout.  
   - Mengecek status code, menampilkan error atau warning jika gagal.

4. **Parsing HTML**  
   - Mengubah HTML menjadi objek `BeautifulSoup` agar bisa diolah fungsi-fungsi detail (`get_detail_*`).

5. **Safe call ke fungsi detail**  
   - Memanggil semua fungsi helper (`get_detail_sekolah`, `get_detail_profil`, `get_detail_ekskul`, dll) dengan **penanganan error** sehingga jika salah satu gagal, proses tetap berjalan.

6. **Gabungkan semua data**  
   - Data dari berbagai fungsi digabungkan menjadi satu dictionary `data_keseluruhan`.  
   - Jika ada data kosong, diberikan **default values** agar struktur data tetap konsisten.

7. **Simpan file JSON**  
   - Folder otomatis dibuat sesuai tipe sekolah.  
   - Data disimpan sebagai JSON dengan encoding `utf-8` dan format rapi (`indent=4`).

8. **Kembalikan data flatten**  
   - Fungsi `flatten_values()` digunakan untuk meratakan semua nilai dari struktur nested menjadi **list satu dimensi**, siap untuk disimpan di Google Spreadsheet menggunakan fungsi `simpan()`.

**Ringkasnya:** fungsi ini melakukan **scraping, parsing, penggabungan data, penyimpanan JSON, dan persiapan data untuk spreadsheet** dalam satu alur terpadu.


In [ ]:
def scrapper(i, nama, url, tipe_sekolah):
    # Pastikan i integer
    try:
        i = int(i)
    except ValueError:
        raise ValueError(f"Parameter 'i' harus integer, tapi dapat: {i}")

    # Persiapan nama file
    nama_file = nama.lower().replace(" ", "_")

    # Siapkan URL
    url_sekolah = url
    url_sanitasi = url_sekolah.replace("Chome", "Csanitasi").replace("profil/", "profil?id=")

    # Ambil halaman
    try:
        res_sekolah = requests.get(url_sekolah, timeout=20)
        res_sanitasi = requests.get(url_sanitasi, timeout=20)
    except requests.RequestException as e:
        print(f"[ERROR] Gagal menghubungi server: {e}")
        return []

    if res_sekolah.status_code != 200:
        print(f"[ERROR] Gagal ambil data sekolah ({res_sekolah.status_code})")
        return []
    if res_sanitasi.status_code != 200:
        print(f"[WARNING] Gagal ambil data sanitasi ({res_sanitasi.status_code})")

    # Parsing HTML
    html_soup_sekolah = BeautifulSoup(res_sekolah.text, "html.parser")
    html_soup_sekolah_sanitasi = BeautifulSoup(res_sanitasi.text, "html.parser") if res_sanitasi.status_code == 200 else None

    # Fungsi helper untuk safe call
    def safe_call(func, *args):
        try:
            return func(*args)
        except Exception as e:
            print(f"[WARNING] {func.__name__} gagal: {e}")
            return {}

    # Ambil data
    data_detail_sekolah = safe_call(get_detail_sekolah, html_soup_sekolah)
    data_detail_profil = safe_call(get_detail_profil, html_soup_sekolah)
    data_detail_ekskul = safe_call(get_detail_ekskul, html_soup_sekolah)
    data_profil_sanitasi = safe_call(get_detail_sanitasi, html_soup_sekolah_sanitasi)
    data_instansi_dan_sekolah = safe_call(get_instansi_dan_sekolah, html_soup_sekolah)
    data_detail = safe_call(get_detail, html_soup_sekolah)
    data_detail_akreditsi = safe_call(get_detail_akreditasi, html_soup_sekolah)
    data_detail_bantuan = safe_call(get_detail_bantuan, html_soup_sekolah)

    # Gabungkan
    data_keseluruhan = {
        "detail_sekolah": data_detail_sekolah,
        "data_profil": data_detail_profil,
        "data_ekskul": data_detail_ekskul,
        "data_sanitasi": data_profil_sanitasi,
        "data_terdekat": data_instansi_dan_sekolah,
    }
    if data_detail == {} :
        data_detail = {"siswa": {
                "siswa": {
                    "tingkat": {
                        "10": 0,
                        "11": 0,
                        "12": 0,
                        "total": 0
                    },
                    "agama": {
                        "laki-laki": {
                            "islam": 0,
                            "kristen": 0,
                            "katholik": 0,
                            "hindu": 0,
                            "budha": 0,
                            "kong hu chu": 0,
                            "kepercayaan": 0,
                            "lainnya": 0,
                            "total": 0
                        },
                        "perempuan": {
                            "islam": 0,
                            "kristen": 0,
                            "katholik": 0,
                            "hindu": 0,
                            "budha": 0,
                            "kong hu chu": 0,
                            "kepercayaan": 0,
                            "lainnya": 0,
                            "total": 0
                        }
                    },
                    "umur": {
                        "< 16 tahun": 0,
                        "16 - 18 tahun": 0,
                        "> 18 tahun": 0,
                        "total": 0
                    },
                    "jenis_kelamin": {
                        "laki-laki": 0,
                        "perempuan": 0,
                        "total": 0
                    }
                },
                "siswa_mengulang": {
                    "tingkat": {
                        "10": 0,
                        "11": 0,
                        "12": 0,
                        "total": 0
                    },
                    "umur": {
                        "< 16 tahun": 0,
                        "16 - 18 tahun": 0,
                        "> 18 tahun": 0,
                        "total": 0
                    },
                    "jenis_kelamin": {
                        "laki-laki": 0,
                        "perempuan": 0,
                        "total": 0
                    }
                },
                "siswa_baru": {
                    "umur": {
                        "< 16 tahun": 0,
                        "16 - 18 tahun": 0,
                        "> 18 tahun": 0,
                        "total": 0
                    },
                    "jenis_kelamin": {
                        "laki-laki": 1,
                        "perempuan": 0,
                        "total": 1
                    }
                },
                "siswa_lulus": {
                    "umur": {
                        "< 16 tahun": 0,
                        "16 - 18 tahun": 0,
                        "> 18 tahun": 0,
                        "total": 0
                    },
                    "jenis_kelamin": {
                        "laki-laki": 0,
                        "perempuan": 0,
                        "total": 0
                    }
                }
            },
            "guru": {
                "guru": {
                    "status": {
                        "pns": 0,
                        "gtt": 0,
                        "gty": 0,
                        "honor": 0,
                        "total": 0
                    },
                    "golongan": {
                        "I": 0,
                        "II": 0,
                        "III": 0,
                        "IV": 0,
                        "Total": 0
                    },
                    "sertifikasi": {
                        "sertifikasi": 0,
                        "belum sertifikasi": 0,
                        "total": 0
                    },
                    "ijazah": {
                        "kurang dari s1": 0,
                        "s1 atau lebih": 0,
                        "data kosong": 0,
                        "total": 0
                    },
                    "umur": {
                        "kurang dari 30 tahun": 0,
                        "31 - 35 tahun": 0,
                        "36 - 40": 0,
                        "41 - 45 tahun": 0,
                        "46 - 50 tahun": 0,
                        "51 - 55 tahun": 0,
                        "lebih dari 55 tahun": 0,
                        "total": 0
                    },
                    "jenis_kelamin": {
                        "laki-laki": 0,
                        "perempuan": 0,
                        "total": 0
                    }
                },
                "tenaga_kependidikan": {
                    "status": {
                        "pns": 0,
                        "honor": 0,
                        "total": 0
                    },
                    "golongan": {
                        "I": 0,
                        "II": 0,
                        "III": 0,
                        "IV": 0,
                        "Total": 0
                    },
                    "ijazah": {
                        "kurang dari s1": 0,
                        "s1 atau lebih": 0,
                        "data kosong": 0,
                        "total": 0
                    },
                    "umur": {
                        "kurang dari 30 tahun": 0,
                        "31 - 35 tahun": 0,
                        "36 - 40": 0,
                        "41 - 45 tahun": 0,
                        "46 - 50 tahun": 8,
                        "51 - 55 tahun": 0,
                        "lebih dari 55 tahun": 0,
                        "total": 0
                    },
                    "jenis_kelamin": {
                        "laki-laki": 0,
                        "perempuan": 0,
                        "total": 0
                    }
                }
            },
            "rombongan_belajar": {
                "tingkat": {
                    "10": 0,
                    "11": 0,
                    "12": 0,
                    "total": 0
                }
            },
            "sarana_prasarana": {
                "ruang_kelas": {
                    "baik": 0,
                    "rusak ringan": 0,
                    "rusak sedang": 0,
                    "rusak berat": 0,
                    "total": 0
                },
                "laboratorium": {
                    "IPA": {
                        "Baik": 0,
                        "Rusak Ringan": 0,
                        "Rusak Sedang": 0,
                        "Rusak Berat": 0,
                        "total": 0
                    },
                    "Bahasa": {
                        "Baik": 0,
                        "Rusak Ringan": 0,
                        "Rusak Sedang": 0,
                        "Rusak Berat": 0,
                        "total": 0
                    },
                    "IPS": {
                        "Baik": 0,
                        "Rusak Ringan": 0,
                        "Rusak Sedang": 0,
                        "Rusak Berat": 0,
                        "total": 0
                    },
                    "Komputer": {
                        "Baik": 0,
                        "Rusak Ringan": 0,
                        "Rusak Sedang": 0,
                        "Rusak Berat": 0,
                        "total": 0
                    },
                    "Total": {
                        "Baik": 0,
                        "Rusak Ringan": 0,
                        "Rusak Sedang": 0,
                        "Rusak Berat": 0,
                        "total": 0
                    }
                },
                "perpustakaan": {
                    "baik": 0,
                    "rusak ringan": 0,
                    "rusak sedang": 0,
                    "rusak berat": 0,
                    "total": 0
                },
                "sanitasi": {
                    "Guru": {
                        "Baik": 0,
                        "Rusak Ringan": 0,
                        "Rusak Sedang": 0,
                        "Rusak Berat": 0,
                        "total": 0
                    },
                    "Siswa": {
                        "Baik": 0,
                        "Rusak Ringan": 0,
                        "Rusak Sedang": 0,
                        "Rusak Berat": 0,
                        "total": 0
                    },
                    "Total": {
                        "Baik": 0,
                        "Rusak Ringan": 0,
                        "Rusak Sedang": 0,
                        "Rusak Berat": 0,
                        "total": 4212
                    }
                }
            }
        }

    for key, value in data_detail.items():
        data_keseluruhan[key] = value

    data_keseluruhan['data_akreditasi'] = data_detail_akreditsi
    data_keseluruhan['data_bantuan'] = data_detail_bantuan

    # Pastikan folder ada
    os.makedirs(f"../hasil/data_profil/{tipe_sekolah}", exist_ok=True)

    # Simpan JSON
    try:
        with open(f"../hasil/data_profil/{tipe_sekolah}/{i}_{nama_file}.json", "w", encoding="utf-8") as file:
            json.dump(data_keseluruhan, file, ensure_ascii=False, indent=4)
    except Exception as e:
        print(f"[ERROR] Gagal menyimpan file JSON: {e}")
        return []

    # Kembalikan data flatten
    return flatten_values(data_keseluruhan)


## Dengan Multi-Worker

Konfigurasi Scraping

Beberapa variabel yang digunakan untuk mengatur proses scraping:

- `baris_awal = 1` → indeks baris awal yang akan di-scrape.
- `baris_akhir = 10` → indeks baris terakhir yang akan di-scrape.
- `tipe_sekolah = "SMA"` → tipe sekolah yang sedang diproses.
- `MAX_WORKER = 5` → jumlah maksimal worker/thread yang akan digunakan untuk scraping paralel.


In [ ]:
baris_awal = 1
baris_akhir = 1
tipe_sekolah = "SMA"

MAX_WORKER = 1

In [ ]:
def proses_baris(baris):
    try:
        # Ambil data link sekolah
        data_sekolah = ambil_baris(sheet, baris)
        i = int(data_sekolah[0])
        nama_sekolah = data_sekolah[2]
        nama_sekolah = nama_sekolah.replace("/", "")
        url_sekolah = data_sekolah[3]

        # Proses scrapping
        hasil = scrapper(i, nama_sekolah, url_sekolah, tipe_sekolah=tipe_sekolah.lower())

        # Menyimpan data
        simpan(sheet, hasil, i)

        return f"[OK] Baris {baris} - {nama_sekolah}"

    except Exception as e:
        return f"[ERROR] Baris {baris} gagal: {e}"


print(f"Scrapping Baris {baris_awal} - {baris_akhir} ...")
# Eksekusi paralel
with ThreadPoolExecutor(max_workers=MAX_WORKER) as executor:
    futures = [executor.submit(proses_baris, baris) for baris in range(baris_awal, baris_akhir + 1)]
    for future in as_completed(futures):
        print(future.result())

Scrapping Baris 1 - 1 ...
[OK] Baris 1 - SMA NEGERI 1 LEMBAH SEULAWAH


**Catatan akhir:**  
Pada akhirnya, proses ini (`scraping_profil`) akan menghasilkan **data lengkap sekolah** dalam Google Spreadsheet.

Google Spreadsheets dapat dilihat pada [data semua sma](https://docs.google.com/spreadsheets/d/1eJ-uylcTjo_OGhcih7pgacc28KCz9Y8bx6tNw1157GM/) pada sheet `"scraping-profil"`

Selanjutnya, data ini dapat digunakan untuk melakukan **scraping Google Maps** untuk mendapatkan citra satelit dan informasi lokasi sekolah.